In [1]:
%pip install geopandas pandas shapely

Note: you may need to restart the kernel to use updated packages.


In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [4]:
# Load the shapefile
landcover = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/landuse/gis_osm_landuse_a_free_1.shp")

# Load your shapefile (replace with your actual file path)
communes = gpd.read_file("C:/Users/HP/OneDrive - Université Mohammed VI Polytechnique/Desktop/UM6P/Internship/The Journey/Data/QGIS/data/populaion_commune/populaion_commune.shp")

In [5]:
# Filter landcover by target fclasses
lc_filtered = landcover[landcover['fclass'].isin(['farmland', 'farmyard', 'vineyard'])].copy()

In [6]:
# Reproject everything to metric CRS (e.g. EPSG:32630 for Morocco)
target_crs = "EPSG:32630"
communes = communes.to_crs(target_crs)
lc_filtered = lc_filtered.to_crs(target_crs)

In [7]:
df = pd.read_csv('dams.csv')

In [8]:
# Load dam points from DataFrame
# Example: df = pd.read_csv("dam_points.csv")  # with 'latitude' and 'longitude'
dam_gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"
).to_crs(target_crs)

In [9]:
# Step 1: Spatial join to assign each dam to a commune
dam_with_commune = gpd.sjoin(dam_gdf, communes, how='left', predicate='within')

In [10]:
# Step 2: For each commune, calculate landcover area by type
# First, calculate geometry area
lc_filtered["area_m2"] = lc_filtered.geometry.area

In [11]:
# Spatial join to match landcover polygons to communes
lc_with_commune = gpd.sjoin(lc_filtered, communes, how="inner", predicate="intersects")

In [12]:

# Group by commune ID and fclass to get total areas
commune_lc_area = lc_with_commune.groupby(
    [lc_with_commune.index_right, 'fclass']
)['area_m2'].sum().unstack(fill_value=0).reset_index()

In [13]:

# Merge back with dam_with_commune using commune index (from sjoin)
dam_with_commune['commune_id'] = dam_with_commune.index_right
result = dam_with_commune.merge(commune_lc_area, left_on='commune_id', right_on='index_right', how='left')

In [14]:
# Fill missing landcover types with 0
for col in ['farmland', 'farmyard', 'vineyard']:
    if col not in result:
        result[col] = 0
    else:
        result[col] = result[col].fillna(0)

In [15]:
# Reproject back to WGS84 for output
result = result.to_crs("EPSG:4326")
result['latitude'] = result.geometry.y
result['longitude'] = result.geometry.x

In [16]:
# Output CSV
result[['latitude', 'longitude', 'farmland', 'farmyard', 'vineyard']].rename(
    columns={
        'farmland': 'farmland_area_m2',
        'farmyard': 'farmyard_area_m2',
        'vineyard': 'vineyard_area_m2'
    }
).to_csv("dam_commune_landuse_areas.csv", index=False)